# 誤差逆伝播法(back-propagation)
***
誤差逆伝播法とは、深層学習においてパラメータを更新する際に出力層から各ノードに対して逆方向に更新量を計算するアルゴリズムのことである。確率的勾配降下法においては、偏微分を繰り返し行うことで傾きを求め、その傾きに学習率をかけたものを重みから引いていくことで最適となる重みを探索を行っていく。ここで、ミニバッチ法といわれる複数データを同時に実行しそれぞれ更新量を求めたものの平均をとる手法をとることで局所的最適解に到達がほぼすべてのケースにおいて適切に設定のもと可能であるとされている。

## Numpyでの誤差逆伝播法の実装について
***
本項目では、Numpyを用いて誤差逆伝播法の実装を目指していく。※あくまで自力での挑戦となるため、これが正しい答えとは限らないため注意  
また、Numpyで自動的に行うようにするためには以下のような機能について検討する必要がある。

1. 層数を取得
2. 指定された活性化関数を取得
3. 学習率の決定
4. 微分を行って勾配を求める
5. 各ノードに対しこれを繰り返す

# 偏微分
***
誤差逆伝播法では、勾配を求めることで重みを更新していく。そこで必須となるのが、微分である。本来であれば、指定するだけで勾配を返してくれる便利なモジュールも沢山あるが、ここではあえて自作することで理解を深めていこう。

ではまず、微分とはどのようなアルゴリズムで行われているか。それは簡単である。**極僅かな間隔での上昇量**を求める。つまり、現在の値からほんの少し、ほぼ0といえるほど増加させた際の上昇量を求めればよい。つまり、式はこうなる。

$$
    \frac{\Delta f(x)}{\Delta x} = \frac{f(x + \Delta x) - f(x)}{\Delta x}
$$

ここで言っていることは 微小空間での上昇量/微小空間　で求めた値が勾配であるということだ。非常に簡単だろう。では、偏微分ではどうだろうか。

In [5]:
# 微分
def numerical_gradient(func,x):
    # 極僅かな上昇量( Δh )
    dh = 1e-4
    grad = np.zeros_like(x) # xと同じ形状の配列を生成
    
    for idx in range(x.size):
        tmp = x[idx]
        # f(x + h)
        x[idx] = tmp + h
        fxh1 = f(x)
        
        # f(x - h)
        x[idx] = tmp - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 -fxh2) / (2*h)
        x[idx] = tmp
    
    return grad

In [6]:
def cross_entropy(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
        batch_size = y.shape[0]
        return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [7]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [1]:
def relu(x):
    return np.maximum(0, x)

In [42]:
def softmax(a):
    y = np.zeros_like(a)
    for i in range(a.shape[0]):
        exp_a = np.exp(a[i])
        sum_exp_a = np.sum(exp_a)

        y[i] = exp_a / sum_exp_a
    
    return y

In [43]:
import numpy as np
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"]= weight_init_std * np.random.randn(hidden_size,output_size)
        self.params["b2"]= np.zeros(output_size)
        
    def predict(self, x):
        W1,W2 = self.params["W1"], self.params["W2"]
        b1,b2 = self.params["b1"], self.params["b2"]
        
        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x):
        y = self.predict(x)
        
        return cross_entropy(y,t)

    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x,t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        
        # 各パラメータごとに勾配を求める
        grads["W1"] = numerical_gradient(loss_w, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_w, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_w, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_w, self.params["b2"])
        
        return grads

In [47]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size = 10)

x = np.random.rand(100,784)
y = net.predict(x)

print(y[1].sum())

1.0


# pythonのlambdaとは
***
二層NNを構築するするクラスTwoLayerNetだが、その中でnumerical_gradientというものがある。これは、勾配を求めるモジュールだが、その中でloss_Wというものを定義しており、その中でもlambdaという得体の知れないものが含まれており、記述もよくわからない感じになっている。これは一体なにをしているのか、具体的な例と共に説明する。

まず、loss_Wはどこで使われているのかだが、numerical_gradientの引数として用いられている。ここで、「このモジュール名もnumerical_gradientじゃね？」と思ったかもしれないが、ここで用いているのは偏微分を行い勾配を求めるものであり、完全に別物である。ややこしいのは参考とした教材の命名規則に文句を言ってほしい。まあそれは置いておくとして、loss_Wは一つ目の引数として指定されている。ここで、少しさかのぼって一つ目の引数は何だったかを見てみると、funcとある。つまりは関数である。これで勘のいいひとは気付いたかもしれないが、ここで言うlambdaは簡単に言うと**入力と返り値のみの関数を簡単に定義できる**ものである。

記述としては  
<pre>
    lambda 引数: 返り値
</pre>

となっており、引数を複数個指定することなども可能である。

In [54]:
# lambdaお試し
plus2 = lambda x: x+2

def plus_2(x):
    return x+2

print(plus2(3),plus_2(3))
# 扱いも関数定義と同じになっている
print(plus_2)
print(plus2)

5 5
<function plus_2 at 0x000001C7CE197C10>
<function <lambda> at 0x000001C7CE197790>


# 実際の誤差逆伝播法について
***
誤差逆伝播法について、やたらと偏微分やら連鎖律という単語が出てきて難しく感じているかもしれないが、実際は非常に単純である。ここでは、実際にどのようにして誤差逆伝播法が実装されるのか、それを色々な例をもとに説明していく。

<img src="./pic/neural_network4.png" width="400">

上の図は、実際のニューラルネットワークの順伝播の動きを表したものである。これは、以下のような図で表すこともできるだろう。

<img src="./pic/back-propagation1.png">

ここで、$ f_1, f_2 $などの〇で囲まれたものは線形変換を、$ a_1, L $などの□で囲まれたものは非線形変換を行う関数を指しているとする。ここまでは通常の順伝播の流れだ。では、以降では出力から逆に見ていき$ w_1, w_2 $の重み更新量を求めていこう。勾配はそれぞれで**偏微分**すればよいから以下となる。

$$
    \begin{eqnarray}
    w_{1} &=& \frac{\Delta ℓ}{\Delta w_{1}} \\
    w_{2} &=& \frac{\Delta ℓ}{\Delta w_{2}} 
    \end{eqnarray}
$$

非常に簡単である。偏微分が分からない人向けに説明すると、関数$ f $が入力$ x $によって値が一意に定まるとき、関数は$ f(x) $と表せるだろう。では、関数$ f $が入力$ x,y $によって値が一意に定まるときはどうだろう。これは$ f(x,y) $という風に表せる。では、$ f(x,y) $について、**xを固定したまま、あるいはyを固定したまま**もう片方の変数をわずかに上昇させたときの出力の上昇が知りたい(勾配を求めたい)場合、2変数関数ではあるが片方の変数について微分を行うことで求めることができる。つまり、深層学習で言うのであれば、出力は様々な重みやバイアスからなる多変数関数なのだから**重み$ w_{11}とw_{23} $の勾配を求めたい**という風になったら、$ w_{11}とw_{23} $でそれぞれ偏微分すればそれぞれの勾配が求まり、学習率と掛けることでそれぞれの重み更新量を求めることができるのだ。これが誤差逆伝播法における損失関数が最小値となる解を求める学習の概念だ。  

しかし、いきなり上の式が解けたら苦労はしない。先ほども言った通り、ニューラルネットワークの出力は多変数関数となっている。それに加え合成関数でもあるため、一度に偏微分しようとしてもとんでもないことになってしまう。そこで、**連鎖律**というものを用いる。連鎖律とは、合成関数の微分をするときに、$ f(y) $において、入力が$ y(w) $によって与えられるときの$ f $の$ w $での微分は

$$
    \frac{\partial f}{\partial w} = 
    \frac{\partial f}{\partial y}
    \frac{\partial y}{\partial w}
$$

と得られるというルールである。また、関数が$ f(y_1,y_2,...,y_k), y_k(w)$と表されるような多変数関数となっている場合は、

$$
    \frac{\partial f}{\partial w} = 
    \sum^{K}_{k=1}
    \frac{\partial f}{\partial y_k}
    \frac{\partial y_k}{\partial w}
$$

という風に繰り返される。ここまでで理解できたという人はごく少数だとは思うが、とりあえず連鎖律を用いて先ほどの$ w_1, w_2 $についての式をもう少し詳しくしてみよう。

$$
    \begin{eqnarray}
    w_{1} &=& \frac{\Delta ℓ}{\Delta w_{2}} 
        = \frac{\partial ℓ}{\partial y}\frac{\partial y}{\partial w_{2}} \\
    w_{2} &=& \frac{\Delta ℓ}{\Delta w_{1}} 
        = \frac{\partial ℓ}{\partial y}\frac{\partial y}{\partial h_{1}}
        \frac{\partial h_{1}}{\partial u_{1}}\frac{\partial u_{1}}{\partial w_{1}} 
    \end{eqnarray}
$$

以上の通りである。ここで、1つ注意してほしいのが、**誤差逆伝播法は出力から見る**ため、出力に近い$ w_{2} $についての偏微分の方が短い。ここで勘のいいひとは気付いたかもしれないが、連鎖律で用いている変数は2つ目の図で囲まれていない変数である。ここで、上式について、それぞれ右から見ていこう。まずは、$ w_{2} $についてだが、$ w_{2} $を少し増加させると$ y $もわずかに増加し、$ y $がわずかに増加すると$ ℓ $もわずかに増加する。よって、$ w_{2} $をわずかに増加させた際の合成関数$ ℓ $における増加量(勾配)が求まるのだ。$ w_{1} $についても同じである。ここで個人的な見解としては、**誤差逆伝播法は出力から反対方向へ更新量を求めるが、その計算の実は順伝播と同じ方向である**ということだ。

ここまでは誤差逆伝播法について解説したが、いよいよ実際にどのように実装するかだが、その前に一つ心にとめておいてほしいのは、**微分をするのは微小空間での上昇量(傾きや勾配と言う)を求めるため**であるということだ。つまり、実際に0に近いごくわずかな空間での上昇量が求まるのであれば**微分を行う必要がない**のだ。深層学習では、用意したモデルにおいてパラメータを変更することが可能(更新とは違うため注意)であるため、重みやバイアスをごくわずかに増加させて順伝播処理させることで勾配を求めることが可能である。ちなみに、これまで説明した偏微分や連鎖律など、実際の数値を用いずに変数のまま解を得ることを**解析的に解く**と言い、今述べたように実際の値を用いて解を得ることを**数値的に解く**という。もうここまででほとんど述べてしまったが、ニューラルネットワークにおける誤差逆伝播法では、数値的に解くことで最適となる重み(解)を求めていく。

余談だが、これまでの説明で「じゃあほぼ0に近い分だけ増加させてやればいいんでしょ？じゃあ$ 10^{-1000000000000} $で求めるわ」と思ったかもしれない。しかし、pythonでは**丸め誤差**と言って、小さすぎると誤差として0として扱われてしまう。その為、多少の誤差は出るが誤差と間違われない程度の値を設定するようにしよう。

# 今後の課題+α
***
今回は、Numpyを用いてニューラルネットワークを構築する方法や、その実態について学ぶことができた。しかし、今回参考としたクラス TwoLayerNetは2層ニューラルネットワークであり、損失関数や活性化関数が既に定まってしまっていた。そのため、今後はそれらを動的に取得しモデルを構築、学習できるようなアルゴリズムについて考えていきたい。実際は、入力された値によって切り替えればよいだけなので、プログラミングについては物量勝負だけになりそうなので、後回しにはなりそうだが。それに加えて、SymPyを用いた数値解析についても行っていきたい。(SymPyはPythonで記号計算を行うためのライブラリ)

今回は、実際に偏微分や他の関数や誤差逆伝播法について理解を深めることができた。連鎖律や偏微分の概念自体は非常に簡単なので、今後の学習についても非常に役立ちそうな経験だ。ここで得られた別の教訓としては**活性化関数は連続であるものを選ぶ**ことがこれらの学習方法(最適化手法)を用いるならば留意しなくてはならないということだろうか。